In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset["classification_yes"].value_counts()

1    249
0    150
Name: classification_yes, dtype: int64

In [6]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'  ]]
dep=dataset[['classification_yes' ]]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],
              'max_features': ['sqrt','log2'],
              'n_estimators':[10,100]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 458, in fit
    for i, t in enumerate(trees)
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\joblib\parallel.py", line 921, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\ELCOT\Anaconda3\lib\site-p

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             verbose=3)

In [10]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [11]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions )

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions,average='weighted')
print("The f1 macro value for parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}: 0.47029200909249863


In [13]:
print(cm)

[[ 0 51]
 [ 0 82]]


In [14]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.62      1.00      0.76        82

    accuracy                           0.62       133
   macro avg       0.31      0.50      0.38       133
weighted avg       0.38      0.62      0.47       133



In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.96987087517934

In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.172516,0.061129,0.007434,0.014868,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.962963,0.981132,0.943396,0.962264,0.981132,0.966177,0.014085,7
1,0.870252,0.104311,0.059449,0.023924,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.962963,0.943396,0.981132,0.981132,0.981132,0.969951,0.015027,4
2,0.109302,0.050432,0.028926,0.017016,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.924528,0.962264,0.962264,1.000000,0.969811,0.028239,5
3,0.735813,0.074480,0.053016,0.018696,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.943396,0.981132,0.962264,0.981132,0.973585,0.019242,3
4,0.125649,0.030397,0.009375,0.018750,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.944444,0.962264,0.962264,0.962264,1.000000,0.966247,0.018233,6
5,0.970712,0.068146,0.059378,0.041223,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.981481,0.962264,0.981132,0.962264,1.000000,0.977428,0.014139,1
6,0.184514,0.051779,0.003125,0.006250,entropy,log2,10,"{'criterion': 'entropy', 'max_features': 'log2...",0.981481,0.943396,0.924528,0.962264,0.981132,0.958560,0.022076,8
7,1.050559,0.236535,0.065172,0.022496,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.981481,0.943396,0.981132,0.962264,1.000000,0.973655,0.019269,2
8,0.153546,0.064862,0.000000,0.000000,log_loss,sqrt,10,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.340374,0.078008,0.000000,0.000000,log_loss,sqrt,100,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
